# plotly: biblioteca para gráficos interactivos

[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gf0657-programacionsig/2024-ii/blob/main/contenido/3/plotly.ipynb)

**NOTA IMPORTANTE**

Por una falla de Jupyter Book (la biblioteca usada para construir el sitio web de este curso), los gráficos en plotly no se están desplegando, pero pueden verse en:

[https://nbviewer.org/github/gf0657-programacionsig/2024-ii/blob/main/contenido/3/plotly.ipynb](https://nbviewer.org/github/gf0657-programacionsig/2024-ii/blob/main/contenido/3/plotly.ipynb)

o en

[https://colab.research.google.com/github/gf0657-programacionsig/2024-ii/blob/main/contenido/3/plotly.ipynb](https://colab.research.google.com/github/gf0657-programacionsig/2024-ii/blob/main/contenido/3/plotly.ipynb)

## Introducción

[Plotly Python](https://plotly.com/python/) es una biblioteca para gráficos interactivos que forma parte del [grupo de bibliotecas de graficación de plotly](https://plotly.com/graphing-libraries/), el cual también incluye bibliotecas para otros lenguajes como R, Julia, F# y MATLAB. plotly fue originalmente escrita en [JavaScript](https://en.wikipedia.org/wiki/JavaScript), por lo que es particularmente adecuada para gráficos interactivos en la Web.

El módulo [`plotly.express`](https://plotly.com/python/plotly-express/) provee más de 30 [funciones](https://plotly.com/python-api-reference/plotly.express.html) para crear diferentes tipos de gráficos y se recomienda como punto de partida para programar los tipos de gráficos estadísticos más comunes.

## Carga

In [78]:
# Carga de plotly express con el alias px
import plotly.express as px

import plotly.graph_objects as go

import matplotlib.pyplot as plt
from matplotlib import ticker

## Tipos de gráficos

En los siguientes ejemplos, se utiliza el conjunto de datos de [países de Natural Earth](https://github.com/gf0657-programacionsig/2024-ii/blob/main/datos/natural-earth/paises.csv), el cual se carga y se configura en el siguiente bloque de código.

In [79]:
import pandas as pd

# Configuración de pandas para mostrar separadores de miles y 2 dígitos decimales
pd.set_option('display.float_format', '{:,.2f}'.format)

# Carga de datos de países en un dataframe
paises = pd.read_csv(
    "https://raw.githubusercontent.com/gf0657-programacionsig/2024-ii/refs/heads/main/datos/natural-earth/paises.csv"
)

# Se usa la columna ADM0_ISO como índice
paises.set_index('ADM0_ISO', inplace=True)

# Despliegue de una muestra aleatoria de 5 filas
paises.sample(5)


,NAME,CONTINENT,REGION_UN,SUBREGION,REGION_WB,ECONOMY,INCOME_GRP,POP_EST,GDP_MD
ADM0_ISO,,,,,,,,,
NIC,Nicaragua,North America,Americas,Central America,Latin America & Caribbean,6. Developing region,4. Lower middle income,"6,545,502.00",12520
PLW,Palau,Oceania,Oceania,Micronesia,East Asia & Pacific,6. Developing region,3. Upper middle income,"18,008.00",268
COD,Dem. Rep. Congo,Africa,Africa,Middle Africa,Sub-Saharan Africa,7. Least developed region,5. Low income,"86,790,567.00",50400
LKA,Sri Lanka,Asia,Asia,Southern Asia,South Asia,6. Developing region,4. Lower middle income,"21,803,000.00",84008
CYP,Cyprus,Asia,Asia,Western Asia,Europe & Central Asia,6. Developing region,2. High income: nonOECD,"1,198,575.00",24948


### Gráficos de dispersión

Un [gráfico de dispersión (en inglés, *scatter plot*)](https://es.wikipedia.org/wiki/Diagrama_de_dispersi%C3%B3n) despliega los valores de dos variables numéricas, como puntos en un sistema de coordenadas. El valor de una variable se despliega en el eje X y el de la otra variable en el eje Y. Variables adicionales pueden ser mostradas mediante atributos de los puntos, tales como su tamaño, color o forma.

En plotly, los gráficos de dispersión se generan con el método [`plotly.express.scatter()`](https://plotly.github.io/plotly.py-docs/generated/plotly.express.scatter.html). Se recomienda revisar el [tutorial](https://plotly.com/python/line-and-scatter/).

#### Ejemplos

##### Población vs PIB

In [80]:
# Subconjunto de países seleccionados
paises_seleccionados = paises.loc[['PAN', 'CRI', 'NIC', 'SLV', 'HND', 'GTM', 'BLZ']]

# Creación del gráfico de dispersión con plotly express
fig = px.scatter(
    paises_seleccionados,
    x='POP_EST',
    y='GDP_MD',
    text=paises_seleccionados.index, # texto sobre cada punto
    title='Relación entre población y producto interno bruto (PIB)',
    labels={
        'POP_EST': 'Población (habitantes)',
        'GDP_MD': 'PIB (millones de dólares)'
    }
)

# Personalización del gráfico
fig.update_traces(marker=dict(color='red'), textposition='top center')
fig.update_layout(
    xaxis_tickformat=',',
    yaxis_tickformat=',',
    xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='lightgray')
)

# Despliegue del gráfico
fig.show()

##### Esperanza de vida vs PIB per cápita

Se agrega la columna `LIFE_EXPECTANCY` (esperanza de vida al nacer) al dataframe `paises`.

In [81]:
# Carga de datos de esperanza de vida al nacer por país
esperanza_vida = pd.read_csv(
    "https://raw.githubusercontent.com/gf0657-programacionsig/2024-ii/refs/heads/main/datos/world-bank/paises-esperanza-vida.csv"
)

# Se usa la columna 'Country Code' como índice
esperanza_vida.set_index('Country Code', inplace=True)

# Reducción de columnas de esperanza_vida
esperanza_vida = esperanza_vida[['2022']]

# Unión de los dataframes paises y esperanza_vida
paises = paises.join(esperanza_vida, how="left")

# Cambio de nombre de la nueva columna
paises.rename(columns={'2022': 'LIFE_EXPECTANCY'}, inplace=True)

Se agrega la columna `GDP_PC` (producto interno bruto per cápita) al dataframe `paises`.

In [82]:
def pib_per_capita(pib, poblacion):
    """
    Retorna el PIB per cápita dados el PIB de un país (en millones de dólares) y su población.
    """

    return (pib * 1000000) / poblacion

# Creación de la columna GDP_PC (PIB per cápita en dólares)
paises['GDP_PC'] = pib_per_capita(paises['GDP_MD'], paises['POP_EST'])

Se genera el gráfico de dispersión.

In [83]:
# Subconjunto de países seleccionados
paises_seleccionados = paises

# Eliminar valores nulos en las columnas 'GDP_MD' y 'POP_EST'
paises = paises.dropna(subset=['GDP_MD', 'POP_EST'])

# Creación del gráfico de dispersión con Plotly Express
fig = px.scatter(
    paises_seleccionados,
    x='GDP_PC',
    y='LIFE_EXPECTANCY',
    color='CONTINENT',    # para colorear los puntos por continente
    title='Relación entre PIB per cápita y esperanza de vida',
    labels={
        'NAME': 'País',
        'GDP_PC': 'PIB per cápita (dólares)',
        'LIFE_EXPECTANCY': 'Esperanza de vida (años)',
        'CONTINENT': 'Continente'
    },
    hover_data={
        'NAME': True,              # para mostrar la columna NAME
        'GDP_PC': ':,.2f',         # formato con dos decimales y separador de miles
        'LIFE_EXPECTANCY': ':.2f', # formato con dos decimales
    }
)

# Personalización del gráfico
fig.update_traces(
    textposition='top center',
    textfont=dict(size=6)
)

# Ajuste del eje x para que comience en 0
x_max = paises_seleccionados['GDP_PC'].max() * 1.05  # Añade un 5% de margen superior
fig.update_xaxes(range=[-2500, x_max])

fig.update_layout(
    xaxis_tickformat=',',
    yaxis_tickformat=',',
    xaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='lightgray'),
    yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='lightgray')
)

# Despliegue del gráfico
fig.show()

#### Ejercicios

1. Programe gráficos de dispersión que muestren la relación entre los siguientes [indicadores del Banco Mundial](https://datos.bancomundial.org/indicador):
- Tasa de alfabetización de adultos y PIB per cápita.

Considere todos los países del mundo y diferentes regiones (continentes, economías, grupos de ingreso, etc.)

### Gráficos de líneas

Un [gráfico de líneas](https://en.wikipedia.org/wiki/Line_chart) muestra información en la forma de puntos de datos, llamados marcadores (*markers*), conectados por segmentos de líneas rectas. Es similar a un gráfico de dispersión pero, además del uso de segmentos de línea, tiene la particularidad de que los datos están ordenados, usualmente con respecto al eje X. Los gráficos de línea son usados frecuentemente para mostrar tendencias a través del tiempo.

En plotly, los gráficos de barras se generan mediante el método [`plotly.express.line()`](https://plotly.github.io/plotly.py-docs/generated/plotly.express.line.html). Se recomienda revisar el [tutorial](https://plotly.com/python/line-charts/).

#### Ejemplos

##### Evolución en el tiempo de la esperanza de vida al nacer

In [84]:
# Carga de datos de esperanza de vida al nacer por país
esperanza_vida = pd.read_csv(
    "https://raw.githubusercontent.com/gf0657-programacionsig/2024-ii/refs/heads/main/datos/world-bank/paises-esperanza-vida.csv"
)

# Filtrar datos de países
paises_seleccionados = esperanza_vida[esperanza_vida['Country Code'].isin(['CRI', 'COL'])]

# Despliegue de los datos de los pa
paises_seleccionados

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,Unnamed: 68
45,Colombia,COL,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,57.13,57.73,58.30,58.89,59.38,59.81,...,76.26,76.47,76.65,76.75,76.75,74.77,72.83,73.66,NaN,NaN
48,Costa Rica,CRI,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,60.41,60.90,61.11,61.62,62.08,62.66,...,79.09,79.46,79.38,79.48,79.43,79.28,77.02,77.32,NaN,NaN


In [85]:
# Seleccionar solo las columnas de años (asumiendo que comienzan desde la columna 5)
# Ajusta el rango según corresponda a tus datos
anios = esperanza_vida.columns[4:68]
paises_seleccionados = paises_seleccionados[['Country Name', 'Country Code'] + list(anios)]

paises_seleccionados

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
45,Colombia,COL,57.13,57.73,58.30,58.89,59.38,59.81,60.20,60.53,...,76.04,76.26,76.47,76.65,76.75,76.75,74.77,72.83,73.66,NaN
48,Costa Rica,CRI,60.41,60.90,61.11,61.62,62.08,62.66,63.37,63.92,...,78.77,79.09,79.46,79.38,79.48,79.43,79.28,77.02,77.32,NaN


In [86]:
# Transformar los datos de formato ancho a largo
datos_largos = paises_seleccionados.melt(
    id_vars=['Country Name', 'Country Code'],
    value_vars=anios,
    var_name='Year',
    value_name='Life Expectancy'
)

datos_largos

,Country Name,Country Code,Year,Life Expectancy
0,Colombia,COL,1960,57.13
1,Costa Rica,CRI,1960,60.41
2,Colombia,COL,1961,57.73
3,Costa Rica,CRI,1961,60.90
4,Colombia,COL,1962,58.30
...,...,...,...,...
123,Costa Rica,CRI,2021,77.02
124,Colombia,COL,2022,73.66
125,Costa Rica,CRI,2022,77.32
126,Colombia,COL,2023,NaN


In [87]:
# Crear el gráfico de líneas
fig = px.line(
    datos_largos,
    x='Year',
    y='Life Expectancy',
    color='Country Name',
    markers=True,
    labels={
        'Year': 'Año',
        'Life Expectancy': 'Esperanza de vida al nacer (años)',
        'Country Name': 'País'
    },
    title='Evolución en el tiempo de la esperanza de vida al nacer'
)

# Personalizar el diseño del gráfico
fig.update_layout(
    legend_title_text='País',
    xaxis=dict(
        showgrid=True,
        gridcolor='LightGrey',
        gridwidth=1
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='LightGrey',
        gridwidth=1
    )
)

# Mostrar el gráfico
fig.show()

#### Ejercicios

1. Agregue más países al gráfico de evolución en el tiempo de la esperanza de vida al nacer.
2. Elabore un gráfico similar para el indicador de mortalidad infantil. Utilice datos de varios países.

### Gráficos de barras

Un [gráfico de barras](https://es.wikipedia.org/wiki/Diagrama_de_barras) se compone de barras rectangulares con longitud proporcional a estadísticas (ej. frecuencias, promedios, mínimos, máximos) asociadas a una variable categórica o discreta. Las barras pueden ser horizontales o verticales y se recomienda que estén ordenadas según su longitud, a menos que exista un orden inherente a la variable (ej. el orden de los días de la semana). Es uno de los tipos de gráficos estadísticos más antiguos y comunes y tiene la ventaja de ser muy fácil de comprender.

En plotly, los gráficos de barras se generan mediante el método [`plotly.express.bar()`](https://plotly.github.io/plotly.py-docs/generated/plotly.express.bar.html). Se recomiendar leer el [tutorial](https://plotly.com/python/bar-charts/).

#### Ejemplos

##### Suma de población por continente

Gráfico de barras verticales.

In [88]:
# Suma de población por continente
poblacion_continente_suma = paises.groupby('CONTINENT')['POP_EST'].sum().sort_values(ascending=False).reset_index()

# Creación de gráfico de barras verticales
fig = px.bar(
    poblacion_continente_suma,
    x='CONTINENT',
    y='POP_EST',
    title='Suma de población por continente',
    labels={
        'CONTINENT': 'Continente',
        'POP_EST': 'Población (habitantes)'
    },
    width=1000,   # Ancho de la figura en píxeles
    height=600    # Alto de la figura en píxeles
)

# Actualizar el formato del eje y evitar notación científica
fig.update_yaxes(tickformat=",d")

# Personalizar el diseño
fig.update_layout(
    title=dict(
        x=0.5,  # Centrar el título
        font=dict(size=20)
    ),
    xaxis_title=dict(
        font=dict(size=16)
    ),
    yaxis_title=dict(
        font=dict(size=16)
    )
)

# Mostrar el gráfico
fig.show()

Para dibujar el mismo gráfico con barras horizontales, debe utilizarse el argumento `orientation=h`.

In [89]:
# Suma de población por continente
poblacion_continente_suma = paises.groupby('CONTINENT')['POP_EST'].sum().sort_values(ascending=True).reset_index()

# Creación de gráfico de barras verticales
fig = px.bar(
    poblacion_continente_suma,
    x='POP_EST',
    y='CONTINENT',
    orientation='h',
    title='Total de población por continente',
    labels={
        'CONTINENT': 'Continente',
        'POP_EST': 'Población (habitantes)'
    },
    width=1000,   # Ancho de la figura en píxeles
    height=600    # Alto de la figura en píxeles
)

# Actualizar el formato del eje y evitar notación científica
fig.update_xaxes(tickformat=",d")

# Personalizar el diseño
fig.update_layout(
    title=dict(
        x=0.5,  # Centrar el título
        font=dict(size=20)
    ),
    xaxis_title=dict(
        font=dict(size=16)
    ),
    yaxis_title=dict(
        font=dict(size=16)
    )
)

# Mostrar el gráfico
fig.show()

##### Suma de población por región y subregión de la ONU

######  Barras apiladas

Con el argumento `color=SUBREGION` puede generarse un gráfico de barras apiladas (en inglés, *stacked*), el cual permite mostrar datos con varios niveles de agrupación.

In [90]:
# Suma de población por región y subregión de la ONU
# Con reset_index() se evita la creación de un índice y 
# así es posible usar más fácilmente todas las columnas en el gráfico
poblacion_region_subregion_suma = paises.groupby(['REGION_UN', 'SUBREGION'])['POP_EST'].sum().reset_index()

# Ordenar las regiones por población total descendente
poblacion_total_region = poblacion_region_subregion_suma.groupby('REGION_UN')['POP_EST'].sum().sort_values(ascending=False).index

# Asignar el orden de las categorías en el eje X
poblacion_region_subregion_suma['REGION_UN'] = pd.Categorical(
    poblacion_region_subregion_suma['REGION_UN'],
    categories=poblacion_total_region,
    ordered=True
)

# Creación del gráfico de barras apiladas
fig = px.bar(
    poblacion_region_subregion_suma,
    x='REGION_UN',
    y='POP_EST',
    color='SUBREGION',
    title='Total de población por región y subregión de la ONU',
    labels={
        'REGION_UN': 'Región',
        'SUBREGION': 'Subregión',
        'POP_EST': 'Población (habitantes)'
    },
    category_orders={'REGION_UN': poblacion_total_region},  # Aplicar el orden de las regiones
    width=1000,   # Ancho de la figura en píxeles
    height=600    # Alto de la figura en píxeles
)

# Actualizar el formato del eje y para evitar notación científica
fig.update_yaxes(tickformat=",d")

# Personalizar el diseño (opcional)
fig.update_layout(
    title=dict(
        x=0.5,  # Centrar el título
        font=dict(size=20)
    ),
    xaxis_title=dict(
        font=dict(size=16)
    ),
    yaxis_title=dict(
        font=dict(size=16)
    ),
    legend_title=dict(
        text='Subregión',
        font=dict(size=16)
    ),
    legend=dict(
        title_font_size=16,
        font_size=14,
        x=1.05,  # Posición horizontal de la leyenda
        y=1      # Posición vertical de la leyenda
    )
)

# Mostrar el gráfico
fig.show()

######  Barras agrupadas

Otra forma de mostrar barras con diferentes niveles de agrupación son las barras agrupadas, con el argumento `stacked=False`.

In [91]:
# Suma de población por región y subregión de la ONU
poblacion_region_subregion_suma = paises.groupby(['REGION_UN', 'SUBREGION'])['POP_EST'].sum().reset_index()

# Ordenar las regiones por población total descendente
poblacion_total_region = poblacion_region_subregion_suma.groupby('REGION_UN')['POP_EST'].sum().sort_values(ascending=False).index

# Asignar el orden de las categorías en el eje X
poblacion_region_subregion_suma['REGION_UN'] = pd.Categorical(
    poblacion_region_subregion_suma['REGION_UN'],
    categories=poblacion_total_region,
    ordered=True
)

# Creación del gráfico de barras agrupadas con Plotly Express
fig = px.bar(
    poblacion_region_subregion_suma,
    x='REGION_UN',
    y='POP_EST',
    color='SUBREGION',
    title='Suma de población por región y subregión de la ONU',
    labels={
        'REGION_UN': 'Región',
        'SUBREGION': 'Subregión',
        'POP_EST': 'Población (habitantes)'
    },
    category_orders={'REGION_UN': poblacion_total_region},  # Aplicar el orden de las regiones
    width=1000,   # Ancho de la figura en píxeles
    height=600    # Alto de la figura en píxeles
)

# Actualizar el formato del eje y para evitar notación científica
fig.update_yaxes(tickformat=",d")

# Personalizar el diseño para barras agrupadas
fig.update_layout(
    barmode='group',  # Establecer el modo de barras a 'group' para barras agrupadas
    title=dict(
        x=0.5,  # Centrar el título
        font=dict(size=20)
    ),
    xaxis_title=dict(
        font=dict(size=16)
    ),
    yaxis_title=dict(
        font=dict(size=16)
    ),
    legend_title=dict(
        text='Subregión',
        font=dict(size=16)
    ),
    legend=dict(
        title_font_size=16,
        font_size=14,
        x=1.05,  # Posición horizontal de la leyenda
        y=1      # Posición vertical de la leyenda
    )
)

# Mostrar el gráfico
fig.show()

#### Ejercicios

1. Programe gráficos de barras para:
- Suma de PIB (`GDP_MD`) por economía (`ECONOMY`).
- Promedio de PIB per cápita por grupo de ingresos (`INCOME_GRP`).

### Otros

#### Climogramas

Un [climograma](https://es.wikipedia.org/wiki/Climograma) es un gráfico estadístico que combina las variables climáticas de temperatura y precitipación a lo largo de un período de tiempo, para una región o lugar específico. Permite visualizar las condiciones climáticas de un área, facilitando el análisis de patrones estacionales y tendencias climáticas.

Un climograma puede tomar varias formas. Una de las más usuales es la de un gráfico de barras combinado con un gráfico de líneas. Las barras representan las unidades de tiempo (ej. meses) y su longitud el valor de una de las variables (ej. precipitación), mientras que la línea muestra los valores de la otra variable (ej. temperatura).

En los siguientes bloque de código se genera un climograma con [datos de temperatura y precipitación de ERA5](https://cds.climate.copernicus.eu/datasets/reanalysis-era5-land-monthly-means?tab=overview). [ERA5](https://www.ecmwf.int/en/forecasts/dataset/ecmwf-reanalysis-v5) es el quinto conjunto de datos de reanálisis atmosférico global producido por el [Centro Europeo de Predicción a Medio Plazo (ECMWF)](https://www.ecmwf.int/). Es una reconstrucción del clima de la Tierra, desde 1950 hasta la actualidad.

Los datos que se utilizan en el climograma corresponden a los promedios mensuales de temperatura a 2 m de la superficie y de precipitación total para todo el territorio de Costa Rica, entre los años 2004 y 2023. Se utilizó una cuadrícula de 0.1 x 0.1 grados decimales.

In [92]:
# Carga de datos de clima
clima = pd.read_csv(
    "https://raw.githubusercontent.com/gf0657-programacionsig/2024-ii/refs/heads/main/datos/era5/temperatura-precipitacion-cri-2004-2023.csv"
)

# Se usa la columna mes como índice
# clima.set_index('mes', inplace=True)

# Despliegue de los datos
clima

,mes,temperatura,precipitacion
0,Enero,22.15,3.98
1,Febrero,22.60,2.92
2,Marzo,23.16,2.73
3,Abril,23.67,4.72
4,Mayo,23.46,11.91
5,Junio,23.12,12.44
6,Julio,22.87,11.75
7,Agosto,22.95,12.03
8,Septiembre,22.91,11.77
9,Octubre,22.59,14.74


In [93]:
# Crear una figura
fig = go.Figure()

# Añadir las barras de precipitación
fig.add_trace(
    go.Bar(
        x=clima['mes'],
        y=clima['precipitacion'],
        name='Precipitación (mm)',
        marker_color='blue',
        yaxis='y1'
    )
)

# Añadir la línea de temperatura
fig.add_trace(
    go.Scatter(
        x=clima['mes'],
        y=clima['temperatura'],
        name='Temperatura (°C)',
        mode='lines+markers',
        marker=dict(color='red'),
        line=dict(color='red'),
        yaxis='y2'
    )
)

# Actualizar el layout para incluir dos ejes Y
fig.update_layout(
    title='Precipitación y Temperatura Promedio Mensual en Costa Rica (2004-2023)',
    xaxis=dict(
        title='Mes',
        tickmode='linear'
    ),
    yaxis=dict(
        title='Precipitación (mm)',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(
        title='Temperatura (°C)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red'),
        overlaying='y',
        side='right'
    ),
    legend=dict(
        x=0.01,
        y=0.99,
        bgcolor='rgba(255,255,255,0)',
        bordercolor="Black",
        borderwidth=1
    ),
    template='plotly_white',
    width=900,
    height=600
)

# Mostrar el gráfico
fig.show()